In [2]:
%reload_ext sql

In [3]:
%sql sqlite:///ecommerce.db

In [ ]:
import pandas as pd

#Load the dataset
df = pd.read_csv('../data/ecommerce_data.csv')

#Use sqlalchemy to push to SQL
from sqlalchemy import create_engine
engine = create_engine('sqlite:///ecommerce.db') #Same db as above

#Send the dataframe to SQL as table
df.to_sql('ecommerce_data', engine, index=False, if_exists='replace')

5000

# Query1: Conversion Rate by Group(Control vs Treatment)
This tells us which group(control or treatment) had a higher percentage of users who made purchase.

In [5]:
%%sql
SELECT
    [group],
    COUNT(*) AS total_users,
    SUM(purchase) AS total_purchases,
    ROUND(AVG(purchase)*100, 2) AS conversion_rate_percent
FROM ecommerce_data
GROUP BY [group];

 * sqlite:///ecommerce.db
Done.


group,total_users,total_purchases,conversion_rate_percent
control,2504,306,12.22
treatment,2496,345,13.82


# Interpretation:
Based on the above results, Treatment group has a higher conversion rate compared to control group.This suggests that the new feature presented to the treatment group is more effective in driving conversions. However this is a descriptive insight - further statistical testing is required to confirm if the difference is significant or could be due to chance.

# Query2: Average Revenue by Group
This query tells us whether the new feature(treatment group) led to higher earnings per user.

In [6]:
%%sql
SELECT
    [group],
    ROUND(AVG(revenue), 2) AS avg_revenue,
    SUM(revenue) AS total_revenue
FROM ecommerce_data
GROUP BY [group];

 * sqlite:///ecommerce.db
Done.


group,avg_revenue,total_revenue
control,6.12,15326.07
treatment,6.87,17139.1


# Interpretation:
The treatment group shows a higher average revenue per user and greater total revenue. This implies the new feature may be more effective not just at converting users but also at increasing their spending. These business impact makes the treatment promising, but there is need to validate the results with statistical tests.

# Query3: Average Session Time by Device Type
The above query tella us if users on mobile or desktop spend more time on the site.

In [7]:
%%sql
SELECT
    device_type,
    COUNT(*) AS users,
    ROUND(AVG(session_duration), 2)
AS avg_session_seconds
FROM ecommerce_data
GROUP BY device_type;

 * sqlite:///ecommerce.db
Done.


device_type,users,avg_session_seconds
desktop,2079,185.31
mobile,2921,181.03


# Interpretation:
Desktop users show longer average sessions which may suggest they are more engaged or they have a better user experience. This insight could guide design decisions or device specific optimization. If the opposite was to be true then it could have suggested that the mobile experience is effective and user base might be more mobile centric.

# Query4: Conversion Rate by Device Type
This query calculates how likely users are to convert (make a purchase) based on their device type.

In [9]:
%%sql
SELECT 
    device_type,
    COUNT(*) AS total_users,
    SUM(purchase) AS total_purchases,
    ROUND(AVG(purchase)*100, 2) AS conversion_rate_percent
FROM ecommerce_data
GROUP BY device_type;

 * sqlite:///ecommerce.db
Done.


device_type,total_users,total_purchases,conversion_rate_percent
desktop,2079,259,12.46
mobile,2921,392,13.42


# Interpretation:
Based on the results of the above query the mobile device type has a better conversion rate as compared to the desktop device type.

# Query5: Revenue Per Session By Group
This query calculates the average revenue per second of session time for each group. It evaluates how effective each group is at generating revenue relative to time spent on the site.

In [10]:
%%sql
SELECT
    [group],
    ROUND(SUM(revenue) / SUM(session_duration), 4) AS revenue_per_second
FROM ecommerce_data
GROUP BY [group];

 * sqlite:///ecommerce.db
Done.


group,revenue_per_second
control,0.0342
treatment,0.0368


# Interpretation:
Treatment group has a higher revenue per second which could suggest a more efficient user experience or stronger conversion funnel.